In [1]:
import pandas as pd
import clean_tweets_dataframe

In [2]:
# Function: takes a csv file path and reads it into a dataframe
def read_final_data(csv_path):
    try:
        df = pd.read_csv(csv_path)
        print("File is successfully read as csv")
        return df
    except FileNotFoundError:
        print("File Not Found!")

In [4]:
# Reading the cleaned and final data "clean_tweets"
df = read_final_data('cleaned_tweets.csv')

File is successfully read as csv


In [5]:
# View overall insights about the columns of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21997 entries, 0 to 21996
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_at          21997 non-null  object 
 1   source              21997 non-null  object 
 2   original_text       21997 non-null  object 
 3   polarity            21997 non-null  float64
 4   subjectivity        21997 non-null  float64
 5   lang                21997 non-null  object 
 6   favorite_count      21997 non-null  int64  
 7   retweet_count       17257 non-null  float64
 8   original_author     21997 non-null  object 
 9   followers_count     21997 non-null  int64  
 10  friends_count       21997 non-null  int64  
 11  possibly_sensitive  0 non-null      float64
 12  hashtags            19138 non-null  object 
 13  user_mentions       18695 non-null  object 
 14  place               12106 non-null  object 
 15  friends_count       21997 non-null  float64
dtypes: f